In [1]:
! mkdir /root/.kaggle
! mv kaggle.json /root/.kaggle

In [2]:
import kaggle

In [3]:
! kaggle datasets download -d bmadushanirodrigo/fracture-multi-region-x-ray-data

Dataset URL: https://www.kaggle.com/datasets/bmadushanirodrigo/fracture-multi-region-x-ray-data
License(s): ODC Public Domain Dedication and Licence (PDDL)
 98% 473M/481M [00:06<00:00, 132MB/s]
100% 481M/481M [00:06<00:00, 74.7MB/s]


In [4]:
! unzip fracture-multi-region-x-ray-data.zip

Streaming output truncated to the last 5000 lines.
  inflating: Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/not fractured/14-rotated2-rotated2-rotated3 (1).jpg  
  inflating: Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/not fractured/14-rotated2-rotated2-rotated3-rotated1 (1).jpg  
  inflating: Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/not fractured/14-rotated2-rotated2-rotated3-rotated1.jpg  
  inflating: Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/not fractured/14-rotated2-rotated2-rotated3.jpg  
  inflating: Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/not fractured/14-rotated2-rotated2.jpg  
  inflating: Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/not fractured/14-rotated2-rotated3 (1).jpg  
  inflating: Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/

In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential

In [6]:
class CustomImageDataGenerator(Sequence):
    def __init__(self, directory, target_size=(150, 150), batch_size=32, class_mode='binary'):
        self.datagen = ImageDataGenerator(rescale=1./255)
        self.generator = self.datagen.flow_from_directory(
            directory,
            target_size=target_size,
            batch_size=batch_size,
            class_mode=class_mode
        )
        self.length = len(self.generator)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        while True:
            try:
                batch = next(self.generator)
                return batch
            except OSError as e:
                print(f"  Skipping corrupted image: {e}")
                continue

train_idg = CustomImageDataGenerator('/content/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train')
val_idg = CustomImageDataGenerator('/content/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test')
test_idg = val_generator = CustomImageDataGenerator('/content/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/val')

Found 9246 images belonging to 2 classes.
Found 506 images belonging to 2 classes.
Found 829 images belonging to 2 classes.


In [7]:
model = Sequential()

In [8]:
model = Sequential()
model.add(Conv2D(64,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 32)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 41472)             0         
                                                                 
 dense (Dense)               (None, 128)              

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.binary_crossentropy,
              metrics = ['acc'])

In [11]:
model.fit(train_idg, validation_data = val_idg, epochs = 10, batch_size = 32)

Epoch 1/10
289/289 [==============================] - ETA: 0s - loss: 0.2922 - acc: 0.8670  Skipping corrupted image: image file is truncated (33 bytes not processed)
  Skipping corrupted image: image file is truncated (10 bytes not processed)
  Skipping corrupted image: image file is truncated (40 bytes not processed)
  Skipping corrupted image: image file is truncated (40 bytes not processed)
289/289 [==============================] - 33s 94ms/step - loss: 0.2922 - acc: 0.8670 - val_loss: 0.1475 - val_acc: 0.9545
Epoch 2/10
277/289 [===========================>..] - ETA: 0s - loss: 0.0473 - acc: 0.9835  Skipping corrupted image: image file is truncated (40 bytes not processed)
  Skipping corrupted image: image file is truncated (1 bytes not processed)
289/289 [==============================] - ETA: 0s - loss: 0.0467 - acc: 0.9838  Skipping corrupted image: image file is truncated (40 bytes not processed)
  Skipping corrupted image: image file is truncated (33 bytes not processed)
  S

In [12]:
model.evaluate(test_idg)

 3/26 [==>...........................] - ETA: 6s - loss: 0.1181 - acc: 0.9896  Skipping corrupted image: image file is truncated (33 bytes not processed)


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


11/26 [===========>..................] - ETA: 4s - loss: 0.1031 - acc: 0.9886  Skipping corrupted image: image file is truncated (1 bytes not processed)
  Skipping corrupted image: image file is truncated (40 bytes not processed)
23/26 [=========================>....] - ETA: 0s - loss: 0.0735 - acc: 0.9877  Skipping corrupted image: image file is truncated (1 bytes not processed)
  Skipping corrupted image: image file is truncated (10 bytes not processed)
  Skipping corrupted image: image file is truncated (33 bytes not processed)
26/26 [==============================] - 7s 266ms/step - loss: 0.0675 - acc: 0.9879


[0.06752641499042511, 0.9879372715950012]

In [13]:
model.save('bone_fracture_dataset.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
